# FIN 514 - Lecture 3 Python Codes
**Spring 2022**

This notebook provides the values for European options, American options and the callable notes from Lecture 3

## Packages and Configurations

The following common packages will be use on this notebook.

* numpy - [https://numpy.org/](https://numpy.org/)
* Pandas - [https://pandas.pydata.org/](https://pandas.pydata.org/)
* matplotlib - [https://matplotlib.org/](https://matplotlib.org/)
* Scipy Statistical functions - [https://docs.scipy.org/doc/scipy/reference/stats.html](https://docs.scipy.org/doc/scipy/reference/stats.html)


In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt

In [72]:
# ENTER INPUT FOR: start_step
N = 100

In [76]:
# ENTER INPUT FOR: S0 = Original Stock Price
S0 = 227.29
# ENTER INPUT FOR: K = Excercise Price of Call Option
K = 193.20
# ENTER INPUT FOR: sigma = Annualized (Future) Volatility of Stock Price Returns
sigma = 0.3573
# ENTER INPUT FOR: r = Annualized Continously Compounded Risk-free Rate
r = 0.048
# ENTER INPUT FOR: T = Time Length of Option in which to Exercise (In Years)
T = 1
# ENTER INPUT FOR: D = proportional dividend
D = 0.0219
# ENTER INPUT FOR: ND = number of dividends and TD = array of dividend dates
ND = 4
TD = np.zeros([ND]) #creates an array with 4 entries
TD = [0.125, 0.375, 0.625, 0.875]

## Stock value tree

In [77]:
def Stock_tree(N, S0, sigma, r, T, D, TD):
    
    stock_value = np.zeros([N+1, N+1])    
    
    delta = T / N
    u = np.exp(r*delta + sigma * (delta)**0.5)
    d = np.exp(r*delta - sigma * (delta)**0.5)
    print("u = ", u, "d=", d)

    # FIRST LET'S BUILD A STOCK PRICE TREE WITH DIVIDENDS  
    # Let's have dividends at grid points rather than times
    jD1 = [i/delta for i in TD]
    jD = [np.ceil(i) for i in jD1]
    print(jD)
    
    stock_value[0,0] = S0
    for j in range (1,N+1):
        stock_value[j, 0] = stock_value[j-1, 0]*d 
        for i in range(1, j+1):    
            stock_value[j, i] = stock_value[j-1, i-1]*u      

    # This adjusts all stock prices for that j by the size of the dividend    
        if j in jD: stock_value[j, :] *= (1-D)
    print(stock_value[3,0], stock_value[4,0], stock_value[50,50])        
    return stock_value

In [78]:
stock = Stock_tree(N, S0, sigma, r, T, D, TD)

u =  1.0368735670785485 d= 0.9653640450761569
[13.0, 38.0, 63.0, 88.0]
204.48134220721911 197.39893565576295 1329.3428558939984


In [79]:
stock

array([[2.27290000e+02, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.19417594e+02, 2.35670993e+02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.11817856e+02, 2.27508303e+02, 2.44361023e+02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [6.57420249e+00, 7.06118777e+00, 7.58424657e+00, ...,
        7.23172790e+03, 0.00000000e+00, 0.00000000e+00],
       [6.34649871e+00, 6.81661679e+00, 7.32155895e+00, ...,
        6.98125010e+03, 7.49838750e+03, 0.00000000e+00],
       [6.12668167e+00, 6.58051675e+00, 7.06796976e+00, ...,
        6.73944783e+03, 7.23867369e+03, 7.77487979e+03]])

In [80]:
stock[N,N]

7774.879794607819

In [56]:

def Lect3_European(S0, K, T, r, sigma, N, stock_value):
    
    # LIST TO SAVE RESULTS
    Euro_result = []
        
    # CREATE TWO DIMENSIONAL ARRAY OF SIZE [N+1,N+1] TO STORE ALL STEPS
    # option_value[N+1, N+1]
    option_value = np.zeros([N+1, N+1])
    
    # FOR LOOP STATEMENT: For a Binomial Tree from start_step to N
    
    Deltat = T / N
    u = np.exp(r*Deltat + sigma*(Deltat)**0.5)
    d = np.exp(r*Deltat - sigma*(Deltat)**0.5)
    q = (np.exp(r * Deltat) - d) / (u - d)
        
    # Start at the last step number because we are going to be moving backwards from step number n to step number 0
    j = N 
    for i in range(0, j+1):    
    # Then, calculate the value of the option at that exact position within the binomial tree
 
        option_value[j, i] = np.maximum(stock_value[j, i] - K, 0)
    
    # Now, lets calculate the option value at each position (i) within the binomial tree at each previous step number (j) until time zero
    for j in range(N-1, -1, -1):

    # Then, create a FOR iteration on the position number (i), from the top position all the way down to the bottom position of 0 (all down jumps)
        for i in range(j, -1, -1):
            
    # Now, calculation the continuation value of the option at that specific position and step number           
            cv = np.exp(-r * Deltat) * (q*option_value[j+1,i+1] + (1-q)* option_value[j+1,i])
            option_value[j, i] = cv
    # RELAY OUTPUTS TO DICTIONARY
    output = {'num_steps': N, 'CRR': option_value[0,0]}
    Euro_result.append(output)
    print(Euro_result)

    return option_value


In [57]:
value = Lect3_European(S0, K, T, r, sigma, N, stock)


[{'num_steps': 50, 'CRR': 4.555653320003178}]


In [66]:

def Lect3_American(S0, K, T, r, sigma, N, stock_value):
    
    # LIST TO SAVE RESULTS
    American_result = []
        
    # CREATE TWO DIMENSIONAL ARRAY OF SIZE [N+1,N+1] TO STORE ALL STEPS
    # option_value[N+1, N+1]
    option_value = np.zeros([N+1, N+1])
    
    # FOR LOOP STATEMENT: For a Binomial Tree from start_step to N
    
    Deltat = T / N
    u = np.exp(r*Deltat + sigma*(Deltat)**0.5)
    d = np.exp(r*Deltat - sigma*(Deltat)**0.5)
    q = (np.exp(r * Deltat) - d) / (u - d)
        
    # Start at the last step number because we are going to be moving backwards from step number n to step number 0
    j = N 
    for i in range(0, j+1):    
    # Then, calculate the value of the option at that exact position within the binomial tree
 
        option_value[j, i] = np.maximum(stock_value[j, i] - K, 0)
    
    # Now, lets calculate the option value at each position (i) within the binomial tree at each previous step number (j) until time zero
    for j in range(N-1, -1, -1):

    # Then, create a FOR iteration on the position number (i), from the top position all the way down to the bottom position of 0 (all down jumps)
        for i in range(j, -1, -1):
            
    # Now, calculation the continuation and exercise values of the option at that specific position and step number
    # the option value is the larger of the two
            cv = np.exp(-r * Deltat) * (q*option_value[j+1,i+1] + (1-q)* option_value[j+1,i])
            ex = stock_value[j,i] - K
            option_value[j, i] = np.maximum(cv,ex)
    # RELAY OUTPUTS TO DICTIONARY
    output = {'num_steps': N, 'CRR': option_value[0,0]}
    American_result.append(output)

    return American_result, option_value


In [70]:

def Lect3_Barrier(S0, K, T, r, sigma, N, stock_value, B):
    
    # LIST TO SAVE RESULTS
    Barrier_result = []
        
    # CREATE TWO DIMENSIONAL ARRAY OF SIZE [N+1,N+1] TO STORE ALL STEPS
    # option_value[N+1, N+1]
    option_value = np.zeros([N+1, N+1])
    
    # FOR LOOP STATEMENT: For a Binomial Tree from start_step to N
    
    Deltat = T / N
    u = np.exp(r*Deltat + sigma*(Deltat)**0.5)
    d = np.exp(r*Deltat - sigma*(Deltat)**0.5)
    q = (np.exp(r * Deltat) - d) / (u - d)
        
    # Start at the last step number because we are going to be moving backwards from step number n to step number 0
    j = N 
    for i in range(0, j+1):    
    # Then, calculate the value of the option at that exact position within the binomial tree
 
        option_value[j, i] = np.maximum(stock_value[j, i] - K, 0)
    
    # Now, lets calculate the option value at each position (i) within the binomial tree at each previous step number (j) until time zero
    for j in range(N-1, -1, -1):

    # Then, create a FOR iteration on the position number (i), from the top position all the way down to the bottom position of 0 (all down jumps)
        for i in range(j, -1, -1):
            
    # Now, calculation the continuation value of the option at that specific position and step number but replace the option
    #value with 0 if the stock price is below the barrier level
            cv = np.exp(-r * Deltat) * (q*option_value[j+1,i+1] + (1-q)* option_value[j+1,i])
            option_value[j, i] = cv
            if stock_value[j, i] < B: option_value[j, i] = 0
    # RELAY OUTPUTS TO DICTIONARY
    output = {'num_steps': N, 'CRR': option_value[0,0]}
    Barrier_result.append(output)

    return Barrier_result


In [95]:
#Coupon dates
Cpn = 0.1185
NC = 12
TC = np.zeros([NC]) #creates an array with 3 entries
TC = list(np.linspace(1/12,1,12))
N = 100
S0 = 227.29
Face = 1000
T = 1
sigma = .3573
r = .048


In [101]:
Deltat = T/N
jc1 = [j/Deltat for j in TC]
[int(j) for j in jc1]

[8, 16, 25, 33, 41, 49, 58, 66, 75, 83, 91, 100]

In [88]:

def ac_option(S0, K, Face, T, r, sigma, N, stock_value, Cpn, NC, TC, tcall):
    
     # LIST TO SAVE RESULTS
    convertible_result = []
        
    # CREATE TWO DIMENSIONAL ARRAY OF SIZE [N+1,N+1] TO STORE ALL STEPS
    # option_value[N+1, N+1]
    option_value = np.zeros([N+1, N+1])
    
    Deltat = T / N
    u = np.exp(r*Deltat+sigma * (Deltat)**0.5)
    d = np.exp(r*Deltat-sigma * (Deltat)**0.5)
    q = (np.exp(r * Deltat) - d) / (u - d)

    #First, let's calculate the coupon dates and accrued interest

    #jC1 are the exact values of j where the coupons are paid
    #jC are the values of j where we first factor in the coupons - 
    #where int will give us the j immediately before or on the coupon/call date
    jc1 = [j/Deltat for j in TC]
    jc = [int(j) for j in jc1]
    jcall = int(tcall/Deltat)
    print("call period starts", jcall)
    
    
    
    j = N
   
    for i in range(0, j+1): 
        if stock[j,i] >= K:
            option_value[j,i] = Face*(1+Cpn/NC)
        else:
            option_value[j,i] = Face/K*stock[j,i] + Face*(Cpn/NC)
            
            
            
            
        option_value[j, i] = np.maximum(stock_value[j, i] - K, Face*(1+Cpn/NC))
                             
    for j in range(N-1, -1, -1):
        if j >= jcall and j in jc: 
            nextcpndate = TC[jc.index(j)]
            prevcpndate = TC[jc.index(j)-1] 
            print("coupon date",j,jc.index(j),nextcpndate,prevcpndate) 
        if j >= jcall: 
            AI = Face*Cpn/NC*(j*Deltat-prevcpndate)/(nextcpndate-prevcpndate)
            print (j, AI)
        for i in range(0, j+1):           
            cont = np.exp(-r * Deltat) * (q * option_value[j + 1, i + 1] + (1-q) * option_value[j + 1, i])
            if j in jc: cont = cont + Face*Cpn/NC*np.exp(-r*(TC[jc.index(j)]-Deltat*j))  
            conv = ratio*stock_value[j,i]
            call = Face + AI
            option_value[j, i] = np.maximum(conv,cont)                           
            if j >= jcall: option_value[j, i] = np.minimum(np.maximum(call,conv),np.maximum(conv,cont))
                    
      
    output = {'num_steps': N, 'Value': option_value[0,0]}
    convertible_result.append(output)

    return convertible_result




In [89]:
value = Lect3_Convertible(S0, 100, 2, T, r, sigma, N, stock, Cpn, NC, TC, tcall)
value

call period starts 25
49 0.575
48 0.5249999999999999
47 0.47500000000000014
46 0.4250000000000001
45 0.37500000000000006
44 0.325
43 0.27499999999999997
42 0.22499999999999992
41 0.17500000000000016
40 0.1250000000000001
39 0.07500000000000007
38 0.025000000000000022
coupon date 37 2 0.75 0.5
37 0.6
36 0.5499999999999999
35 0.5000000000000002
34 0.4500000000000001
33 0.4000000000000001
32 0.35000000000000003
31 0.3
30 0.24999999999999994
29 0.1999999999999999
28 0.15000000000000013
27 0.10000000000000009
26 0.050000000000000044
coupon date 25 1 0.5 0.25
25 0.625


[{'num_steps': 50, 'Value': 100.06141753314701}]